# Kryptologie Lab - Übung 05

In [1]:
from tqdm.notebook import tqdm

In [2]:
sbox = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]
perm = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

In [3]:
def sbox_block(value, inverse=False):
    if inverse:
        return sbox.index(value)
    else: 
        return sbox[value]

def executeSbox(value,inverse=False):
    block0 = sbox_block(value & 0xF, inverse)
    block1 = sbox_block((value & 0xF0) >> 4, inverse)
    block2 = sbox_block((value & 0xF00) >> 8, inverse)
    block3 = sbox_block((value & 0xF000) >> 12, inverse)
    return (block3 << 12) | (block2 << 8) | (block1 << 4) | block0

def permutation(value,inverse=False):
    output = 0
    for i in range(1, 16):
        bit = value & 0x1
        output = output | (bit << (perm[i] - 1))
        value = value >> 1
    return output

def addRoundKey(value, key):
    return value ^ key

In [4]:
def encrypt(value, key):
    for i in range(3):
        value = addRoundKey(value, key)
        value = executeSbox(value)
        value = permutation(value)
    value = addRoundKey(value, key)
    value = executeSbox(value)
    value = addRoundKey(value, key)
    return value

In [5]:
for i in range(0, 16):
    encrypted = sbox_block(i)
    decrypted = sbox_block(encrypted, inverse=True)
    assert i == decrypted

value = 0xABCD
encrypted = executeSbox(value)
decrypted = executeSbox(encrypted, inverse=True)
assert value == decrypted

encrypt(0xABCD, 0xFABE)

50256

## Lineare Kryptonanalyse

In [6]:
import random

In [7]:
def getBit(val, position):
    return (val >> position) & 0x1

def expandKey(subkey):
    return ((subkey & 0xF) << 4) ^ ((subkey & 0xF0) << 8)

def getProbabilityOfKeys(pairs):
    possibleKeys = dict()
    # iterate possible keys
    for subkey in range(0xFF+1):
        key = expandKey(subkey)
        hits = 0
        # iterate text-cipher pairs
        for pair in pairs:
            x = pair[0]
            y = pair[1]
            v4 = addRoundKey(y, key)
            u4 = executeSbox(v4, inverse=False)
            approx = getBit(x,4) ^ getBit(x,6) ^ getBit(x,7) ^ getBit(u4,5) ^ getBit(u4,7) ^ getBit(u4,13) ^ getBit(u4,15)
            if approx == 0:
                hits += 1
        possibleKeys[subkey] = hits/len(pairs)
    return possibleKeys

def getBestSubKeyOrder(probabilities):
    for data in probabilities.items():
        probabilities.update({data[0]: abs(data[1] - 0.5)})
    return sorted(probabilities.items(), key=lambda item: item[1], reverse=True)

In [24]:
key = 0xFFAE
amount = 12000

M = []
for i in tqdm(range(amount)):
    input = random.getrandbits(16)
    M.append((input, encrypt(input, key)))

In [25]:
probabilities = getProbabilityOfKeys(M)
subkeysInOrderWithProbability = getBestSubKeyOrder(probabilities)
print(subkeysInOrderWithProbability)
subkeysInOrder = [entry[0] for entry in subkeysInOrderWithProbability]
print(subkeysInOrder)

[(83, 0.015249999999999986), (168, 0.014416666666666633), (86, 0.013583333333333336), (19, 0.01358333333333328), (173, 0.013083333333333336), (163, 0.012916666666666687), (227, 0.012750000000000039), (88, 0.011916666666666687), (44, 0.011749999999999983), (118, 0.011666666666666714), (93, 0.011583333333333279), (33, 0.010749999999999982), (172, 0.010249999999999981), (228, 0.010000000000000009), (232, 0.009749999999999981), (199, 0.009666666666666712), (4, 0.009666666666666657), (225, 0.009416666666666629), (108, 0.009333333333333305), (91, 0.009083333333333332), (224, 0.008916666666666684), (85, 0.00883333333333336), (123, 0.00883333333333336), (125, 0.00883333333333336), (246, 0.008750000000000036), (166, 0.008583333333333332), (36, 0.008500000000000008), (212, 0.008416666666666683), (28, 0.008416666666666628), (131, 0.008416666666666628), (198, 0.00833333333333336), (134, 0.00824999999999998), (87, 0.007916666666666683), (103, 0.007833333333333359), (64, 0.007583333333333386), (82, 

## Brutforce in better order

In [26]:
countKeyTries = 0
cracked = False
for key in tqdm(subkeysInOrder):
    countKeyTries += 1
    for brutforceSubkey in range(0xFF+1):
        expandedKey = expandKey(key)
        testKey = (brutforceSubkey & 0xF) ^ ((brutforceSubkey & 0xF0) << 4) ^ expandedKey
        correct = 0
        for m in M[0:10]:
            cipher = encrypt(m[0], testKey)
            if cipher == m[1]:
                correct += 1
        if correct == 10:
            print("Key: ", hex(testKey))
            print("Cracked after ", countKeyTries, " tries")
            cracked = True
            break
    if cracked:
        break       

Key:  0xffae
Cracked after  78  tries

